In [1]:
import glob
from PIL import Image
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from datetime import date
from datetime import datetime

D:\programming\Anaconda1\envs\NN\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


In [2]:
import tensorflow as tf
import numpy as np
import argparse
import math
import os
from matplotlib.image import imread
from scipy.misc.pilutil import imresize, imsave
import matplotlib.pyplot as plt
import cv2

In [3]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Reshape, Flatten, Dropout, Input
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam, SGD
from keras.datasets import mnist
from keras import initializers

import numpy as np
import random

%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
def build_generator():
    
        model = Sequential()
        model.add(Dense(128 * 16 * 16, input_dim = latent_dim))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        model.add(Reshape((16, 16, 128)))
        model.add(Conv2DTranspose(128, filter_size_g, strides=(4,4), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        '''model.add(Conv2DTranspose(64, filter_size_g, strides=(2,2), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())'''
        model.add(Conv2DTranspose(32, filter_size_g, strides=(2,2), padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU())
        model.add(Conv2DTranspose(img_channels, filter_size_g, strides=(2,2), padding='same'))
        model.add(Activation("tanh"))

        model.summary()

        return model

In [ ]:
def build_discriminator():

        model = Sequential()
        model.add(Conv2D(64, kernel_size=filter_size_d, strides = (2,2), input_shape=img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(128, kernel_size=filter_size_d, strides = (2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(256,  kernel_size=filter_size_d, strides = (2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(512, kernel_size=filter_size_d, strides = (2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(512, kernel_size=filter_size_d, strides = (2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(512, kernel_size=filter_size_d, strides = (2,2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Flatten())
        model.add(Dense(1))
        model.add(Activation('sigmoid'))

        model.summary()

        return model

In [ ]:
def generator_containing_discriminator(g, d):
    noise = Input(shape=(latent_dim,))
    img = g(noise)
    d.trainable = False
    #d_label.trainable = False
    valid = d(img)
    
    
    return Model(noise, valid)

In [ ]:
def save_imgs(epoch, gen):
        count = 5
        noise = np.random.uniform(-1, 1, size=(count, latent_dim))
        gen_imgs = gen.predict(noise)

        gen_imgs = 127.5 * gen_imgs + 127.5
        
        if gen_imgs.shape[3] == 1:
            gen_imgs = gen_imgs[:,:,:,0]
        
        for i in range(count):
            cv2.imwrite(os.getcwd() + '\\generated\\epoch%d_%d.png' % (epoch, i), gen_imgs[i])

In [ ]:
def get_images(indices, data, batch_size):
    X_train = np.zeros((batch_size, img_rows, img_cols, img_channels))
    for i in range(batch_size):
        if color_mode == 'grayscale':
            temp_img = cv2.imread(data[indices[i]], 0)
            X_train[i,:,:,0] = temp_img
        else:
            temp_img = cv2.imread(data[indices[i]])
            X_train[i] = temp_img
    X_train = (X_train - 127.5)/127.5
    return X_train

In [ ]:
def get_images_classes(batch_size, data):
    X_train = np.zeros((batch_size, img_rows, img_cols, img_channels))
    y_labels = np.zeros(batch_size)
    choice_arr = np.random.randint(0, len(data), batch_size)
    for i in range(batch_size):
        rand_number = np.random.randint(0, len(data[choice_arr[i]]))
        temp_img = cv2.imread(data[choice_arr[i]][rand_number])
        
        
        X_train[i] = temp_img
        y_labels[i] = choice_arr[i]
    

    X_train = (X_train - 127.5)/127.5
    return X_train, y_labels

In [ ]:
def get_images_one_class(batch_size, data, class_target):
    X_train = np.zeros((batch_size, img_rows, img_cols, img_channels))
    y_labels = np.zeros(batch_size) + class_target
    '''choice_arr = np.random.randint(0, len(data[class_target]), batch_size)
    for i in range(batch_size):
        temp_img = cv2.imread(data[class_target][choice_arr[i]])

        X_train[i] = temp_img
        y_labels[i] = choice_arr[i]'''
    
    for i in range(batch_size):
        temp_img = cv2.imread(os.getcwd() + '/new256_images/abstraktnyy-ekspressionizm/303.png')
        
        X_train[i] = temp_img
        y_labels[i] = 0
    X_train = (X_train - 127.5)/127.5
    return X_train, y_labels

In [ ]:
def combine_images(generated_images):
    num = generated_images.shape[0]
    width = int(math.sqrt(num))
    height = int(math.ceil(float(num)/width))
    shape = generated_images.shape[1:3]
    image = np.zeros((height*shape[0], width*shape[1], img_channels),
                     dtype=generated_images.dtype)
    for index, img in enumerate(generated_images):
        i = int(index/width)
        j = index % width
        image[i*shape[0]:(i+1)*shape[0], j*shape[1]:(j+1)*shape[1]] = \
            img[:, :, :,]
    return image

In [ ]:
def get_data():
    styles_folder = os.listdir(path=os.getcwd() + "\\new256_images\\")
    num_styles = len(styles_folder)
    data = []
    for i in range(num_styles):
        data.append(glob.glob(os.getcwd() + '\\new256_images\\' + styles_folder[i] + '\\*'))
    return data, num_styles

In [ ]:
def train_another(epochs = 100, BATCH_SIZE = 4):

    data, num_styles = get_data()
    
    generator = build_generator()
    discriminator = build_discriminator()
    
    discriminator.compile(loss=losses[0], optimizer=d_optim)
    #d_label.compile(loss=losses[1], optimizer=d_optim)
    generator.compile(loss='binary_crossentropy', optimizer=g_optim)
    
    dcgan = generator_containing_discriminator(generator, discriminator)
    
    
    dcgan.compile(loss=losses[0], optimizer=g_optim)
    
    discriminator.trainable = True
    #d_label.trainable = True
    
    
    '''(X_train, y_train), (X_test, y_test) = mnist.load_data()

    X_train = (X_train.astype(np.float32) - 127.5)/127.5
    X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))'''
    
    
    for epoch in range(epochs):
        for index in range(int(15000/BATCH_SIZE)):
            noise = np.random.normal(0, 1, (BATCH_SIZE, latent_dim))
                
            #real_images, real_labels = get_images_classes(batch_size=BATCH_SIZE, data = data)
            
            label = index % num_styles
            
            real_images, real_labels = get_images_one_class(BATCH_SIZE, data, label)
            
            real_images += np.random.normal(size = img_shape, scale= 0.1)
            
            generated_images = generator.predict(noise)
            
            if index % 2 == 0:
                X = real_images
                real_labels = real_labels - 0.1 + np.random.rand(BATCH_SIZE)*0.2
                y_classif = keras.utils.to_categorical(np.zeros(BATCH_SIZE) + label, num_styles)
                y = 0.8 + np.random.rand(BATCH_SIZE)*0.2
                
                d_loss = []
                d_loss.append(discriminator.train_on_batch(X, y))
                #discriminator.trainable = False
                #d_loss.append(d_label.train_on_batch(X, y_classif))
                print("epoch %d batch %d d_loss : %f" % (epoch, index, d_loss[0]))
            else:
                X = generated_images
                y = np.random.rand(BATCH_SIZE) * 0.2
                d_loss = discriminator.train_on_batch(X, y)
                print("epoch %d batch %d d_loss : %f" % (epoch, index, d_loss))
            
            
            noise = np.random.normal(0, 1, (BATCH_SIZE, latent_dim))
            
            discriminator.trainable = False
            #d_label.trainable = False
            
            y_classif = keras.utils.to_categorical(np.zeros(BATCH_SIZE) + 0.5, num_styles)
            y = np.random.rand(BATCH_SIZE) * 0.3
            
            g_loss = dcgan.train_on_batch(noise, y)
            
            #d_label.trainable = True
            discriminator.trainable = True
            
            print("epoch %d batch %d g_loss : %f" % (epoch, index, g_loss))
            
            if index % 50 == 0:
                        image = combine_images(generated_images)
                        image = image*127.5+127.5
                        cv2.imwrite(
                            os.getcwd() + '\\generated\\epoch%d_%d.png' % (epoch, index), image)
                        image = combine_images(real_images)
                        image = image*127.5+127.5
                        cv2.imwrite(
                            os.getcwd() + '\\generated\\epoch%d_%d_data.png' % (epoch, index), image)
                        
        if epoch % 5 == 0:
            
            
            date_today = date.today()
            
            
            
            month, day = date_today.month, date_today.day
            
            # Генерируем описание модели в формате json
            d_json = discriminator.to_json()
            # Записываем модель в файл
            json_file = open(os.getcwd() + "%d.%d dis_model.json" % (day, month), "w")
            json_file.write(d_json)
            json_file.close()
            
            '''# Генерируем описание модели в формате json
            d_l_json = d_label.to_json()
            # Записываем модель в файл
            json_file = open(os.getcwd() + "%d.%d dis_label_model.json" % (day, month), "w")
            json_file.write(d_l_json)
            json_file.close()'''
            
            # Генерируем описание модели в формате json
            gen_json = generator.to_json()
            # Записываем модель в файл
            json_file = open(os.getcwd() + "%d.%d gen_model.json" % (day, month), "w")
            json_file.write(gen_json)
            json_file.close()
            
            discriminator.save_weights(os.getcwd() + '%d.%d dis_weights.h5' % (day, month))
            #d_label.save_weights(os.getcwd() + '%d.%d dis_label_weights.h5' % (day, month))
            generator.save_weights(os.getcwd() + '%d.%d gen_weights.h5' % (day, month))

In [ ]:
img_rows = 256
img_cols = 256
img_channels = 3
img_shape = (img_rows, img_cols, img_channels)
latent_dim = 100
filter_size_g = (5,5)
filter_size_d = (5,5)
d_strides = (2,2)

color_mode = 'rgb'

losses = ['binary_crossentropy', 'categorical_crossentropy']


#g_optim = SGD(lr = 0.001, momentum=0.9, nesterov=True)
#d_optim = SGD(lr = 0.00025, momentum=0.9, nesterov=True)
g_optim = Adam(0.0002, beta_2 = 0.5)
d_optim = Adam(0.00015, beta_2 = 0.5)

In [ ]:
train_another(1000, 25)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32768)             3309568   
_________________________________________________________________
batch_normalization_1 (Batch (None, 32768)             131072    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32768)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 64, 64, 128)       409728    
_________________________________________________________________
batch_normalization_2 (Batch (None, 64, 64, 128)       512       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 64, 64, 128)       0         
__________

epoch 0 batch 47 g_loss : 1.169052
epoch 0 batch 48 d_loss : 1.606939
epoch 0 batch 48 g_loss : 1.558531
epoch 0 batch 49 d_loss : 0.961804
epoch 0 batch 49 g_loss : 1.429942
epoch 0 batch 50 d_loss : 1.437460
epoch 0 batch 50 g_loss : 1.019150
epoch 0 batch 51 d_loss : 0.972357
epoch 0 batch 51 g_loss : 1.133094
epoch 0 batch 52 d_loss : 1.556081
epoch 0 batch 52 g_loss : 0.994541
epoch 0 batch 53 d_loss : 0.951756
epoch 0 batch 53 g_loss : 1.198649
epoch 0 batch 54 d_loss : 0.841061
epoch 0 batch 54 g_loss : 0.916904
epoch 0 batch 55 d_loss : 0.705350
epoch 0 batch 55 g_loss : 1.263614
epoch 0 batch 56 d_loss : 1.408950
epoch 0 batch 56 g_loss : 0.980341
epoch 0 batch 57 d_loss : 0.708519
epoch 0 batch 57 g_loss : 0.915262
epoch 0 batch 58 d_loss : 0.368338
epoch 0 batch 58 g_loss : 0.900022
epoch 0 batch 59 d_loss : 0.592485
epoch 0 batch 59 g_loss : 0.734335
epoch 0 batch 60 d_loss : 0.910416
epoch 0 batch 60 g_loss : 0.958834
epoch 0 batch 61 d_loss : 0.627946
epoch 0 batch 61 g_l

epoch 0 batch 163 d_loss : 0.335612
epoch 0 batch 163 g_loss : 0.433610
epoch 0 batch 164 d_loss : 0.591455
epoch 0 batch 164 g_loss : 0.432603
epoch 0 batch 165 d_loss : 0.343181
epoch 0 batch 165 g_loss : 0.456221
epoch 0 batch 166 d_loss : 0.542898
epoch 0 batch 166 g_loss : 0.394333
epoch 0 batch 167 d_loss : 0.361306
epoch 0 batch 167 g_loss : 0.417765
epoch 0 batch 168 d_loss : 0.375572
epoch 0 batch 168 g_loss : 0.458156
epoch 0 batch 169 d_loss : 0.330279
epoch 0 batch 169 g_loss : 0.441799
epoch 0 batch 170 d_loss : 0.312093
epoch 0 batch 170 g_loss : 0.450540
epoch 0 batch 171 d_loss : 0.318149
epoch 0 batch 171 g_loss : 0.471840
epoch 0 batch 172 d_loss : 0.348950
epoch 0 batch 172 g_loss : 0.465299
epoch 0 batch 173 d_loss : 0.372780
epoch 0 batch 173 g_loss : 0.444903
epoch 0 batch 174 d_loss : 0.714332
epoch 0 batch 174 g_loss : 0.430066
epoch 0 batch 175 d_loss : 0.334759
epoch 0 batch 175 g_loss : 0.464739
epoch 0 batch 176 d_loss : 0.560411
epoch 0 batch 176 g_loss : 0

epoch 0 batch 277 d_loss : 0.319637
epoch 0 batch 277 g_loss : 0.449203
epoch 0 batch 278 d_loss : 0.331942
epoch 0 batch 278 g_loss : 0.416254
epoch 0 batch 279 d_loss : 0.356526
epoch 0 batch 279 g_loss : 0.474385
epoch 0 batch 280 d_loss : 0.361026
epoch 0 batch 280 g_loss : 0.627418
epoch 0 batch 281 d_loss : 0.367932
epoch 0 batch 281 g_loss : 0.478966
epoch 0 batch 282 d_loss : 0.366508
epoch 0 batch 282 g_loss : 0.450413
epoch 0 batch 283 d_loss : 0.324073
epoch 0 batch 283 g_loss : 0.485460
epoch 0 batch 284 d_loss : 0.361188
epoch 0 batch 284 g_loss : 0.473356
epoch 0 batch 285 d_loss : 0.319157
epoch 0 batch 285 g_loss : 0.552455
epoch 0 batch 286 d_loss : 0.271084
epoch 0 batch 286 g_loss : 0.594261
epoch 0 batch 287 d_loss : 0.327083
epoch 0 batch 287 g_loss : 0.466235
epoch 0 batch 288 d_loss : 0.317309
epoch 0 batch 288 g_loss : 0.548202
epoch 0 batch 289 d_loss : 0.411128
epoch 0 batch 289 g_loss : 0.468857
epoch 0 batch 290 d_loss : 0.325043
epoch 0 batch 290 g_loss : 0

epoch 0 batch 391 d_loss : 0.348143
epoch 0 batch 391 g_loss : 0.391426
epoch 0 batch 392 d_loss : 0.333663
epoch 0 batch 392 g_loss : 0.388624
epoch 0 batch 393 d_loss : 0.350288
epoch 0 batch 393 g_loss : 0.442871
epoch 0 batch 394 d_loss : 0.332546
epoch 0 batch 394 g_loss : 0.489157
epoch 0 batch 395 d_loss : 0.315403
epoch 0 batch 395 g_loss : 0.435944
epoch 0 batch 396 d_loss : 0.349906
epoch 0 batch 396 g_loss : 0.465813
epoch 0 batch 397 d_loss : 0.351265
epoch 0 batch 397 g_loss : 0.402506
epoch 0 batch 398 d_loss : 0.352218
epoch 0 batch 398 g_loss : 0.424946
epoch 0 batch 399 d_loss : 0.364568
epoch 0 batch 399 g_loss : 0.468903
epoch 0 batch 400 d_loss : 0.340590
epoch 0 batch 400 g_loss : 0.404624
epoch 0 batch 401 d_loss : 0.308184
epoch 0 batch 401 g_loss : 0.420483
epoch 0 batch 402 d_loss : 0.287643
epoch 0 batch 402 g_loss : 0.452018
epoch 0 batch 403 d_loss : 0.320998
epoch 0 batch 403 g_loss : 0.475403
epoch 0 batch 404 d_loss : 0.384224
epoch 0 batch 404 g_loss : 0

epoch 0 batch 505 d_loss : 0.307939
epoch 0 batch 505 g_loss : 0.415535
epoch 0 batch 506 d_loss : 0.310187
epoch 0 batch 506 g_loss : 0.477607
epoch 0 batch 507 d_loss : 0.349847
epoch 0 batch 507 g_loss : 0.416318
epoch 0 batch 508 d_loss : 0.326241
epoch 0 batch 508 g_loss : 0.436590
epoch 0 batch 509 d_loss : 0.341144
epoch 0 batch 509 g_loss : 0.494951
epoch 0 batch 510 d_loss : 0.355773
epoch 0 batch 510 g_loss : 0.444476
epoch 0 batch 511 d_loss : 0.330859
epoch 0 batch 511 g_loss : 0.408109
epoch 0 batch 512 d_loss : 0.353938
epoch 0 batch 512 g_loss : 0.409421
epoch 0 batch 513 d_loss : 0.370314
epoch 0 batch 513 g_loss : 0.430715
epoch 0 batch 514 d_loss : 0.310500
epoch 0 batch 514 g_loss : 0.437720
epoch 0 batch 515 d_loss : 0.363692
epoch 0 batch 515 g_loss : 0.461375
epoch 0 batch 516 d_loss : 0.379626
epoch 0 batch 516 g_loss : 0.479671
epoch 0 batch 517 d_loss : 0.353021
epoch 0 batch 517 g_loss : 0.408762
epoch 0 batch 518 d_loss : 0.305863
epoch 0 batch 518 g_loss : 0

In [ ]:
current_moment = datetime.now()

day, month = str(current_moment.day), str(current_moment.month)
hour, minute = str(current_moment.hour), str(current_moment.minute)

with open(os.getcwd() + '/Changes.txt', 'a') as f:
    clock =  (day + '.' + month + '    ' + hour + ':' + minute)
    f.write(clock + '...........g_optim -> 0.0003, d_optim->0.0004' + '\n')